## COLUMN TRANSFORMER

Since in one hot encoding hamle euta column lai split garera different column ma lageko them aba edti column transformer use nagarne ho bhane split garerra stack garne bela hstack use garera stack garnu parxa it will be a little hectic so instead we can use column transformer to make things easy 

In [4]:
import numpy as np
import pandas as pd

In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [8]:
df = pd.read_csv('covid_toy.csv')

In [10]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [20]:
df['gender'].value_counts()

gender
Female    59
Male      41
Name: count, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train , y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

### gender , city --- Nominal Categorical (one hot encoding)
### cough ---- ordinal categorical data 

In [25]:
X_train

,age,gender,fever,cough,city
51,11,Female,100.0,Strong,Kolkata
93,27,Male,100.0,Mild,Kolkata
75,5,Male,102.0,Mild,Kolkata
88,5,Female,100.0,Mild,Kolkata
18,64,Female,98.0,Mild,Bangalore
...,...,...,...,...,...
0,60,Male,103.0,Mild,Kolkata
73,34,Male,98.0,Strong,Kolkata
95,12,Female,104.0,Mild,Bangalore
16,69,Female,103.0,Mild,Kolkata


## Without using Column Transformer

In [34]:
# handling missing values 
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])
X_train_fever.shape

(80, 1)

In [36]:
#ordinal encoding using ordinal encoder
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [54]:
ohe = OneHotEncoder(drop='first', sparse_output=False)

# Fit only on training data
X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])

# Transform test data using the same fitted encoder
X_test_gender_city = ohe.fit_transform(X_test[['gender', 'city']])

X_train_gender_city.shape



(80, 4)

In [60]:
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values
X_train_age.shape

(80, 1)

In [66]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
X_train_transformed.shape

(80, 7)

## Column Transformer

In [83]:
from sklearn.compose import ColumnTransformer

In [91]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [95]:
transformer.fit_transform(X_train).shape

(80, 7)

In [97]:
transformer.transform(X_test).shape

(20, 7)